In [34]:
from nltk.corpus import names
import nltk

In [35]:
labeled_names = ([(name, 'male') for name in names.words('male.txt')] + [(name, 'female') for name in names.words('female.txt')])
import random
random.shuffle(labeled_names)

In [36]:
def gender_features2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

In [37]:
featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]

In [38]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
classifier.classify(gender_features2('Neo'))
print(nltk.classify.accuracy(classifier, test_set))

0.752


In [39]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'            female : male   =     38.1 : 1.0
             last_letter = 'k'              male : female =     32.1 : 1.0
             last_letter = 'p'              male : female =     17.7 : 1.0
             last_letter = 'f'              male : female =     16.1 : 1.0
             last_letter = 'v'              male : female =     11.3 : 1.0


In [40]:
from nltk.corpus import movie_reviews

In [41]:
documents = [(list(movie_reviews.words(fileid)), category)
...              for category in movie_reviews.categories()
...              for fileid in movie_reviews.fileids(category)]

In [43]:
random.shuffle(documents)

In [44]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [45]:
print(document_features(movie_reviews.words('pos/cv957_8737.txt'))) 

contains(train)': True, 'contains(saving)': False, 'contains(share)': False, 'contains(image)': False, 'contains(discovers)': False, 'contains(normal)': False, 'contains(cross)': False, 'contains(fox)': False, 'contains(returns)': False, 'contains(adult)': False, 'contains(adds)': False, 'contains(answer)': False, 'contains(adventure)': False, 'contains(lame)': False, 'contains(male)': False, 'contains(odd)': False, 'contains(singer)': False, 'contains(deserves)': False, 'contains(gore)': False, 'contains(states)': False, 'contains(include)': False, 'contains(equally)': False, 'contains(months)': False, 'contains(barely)': False, 'contains(directors)': False, 'contains(introduced)': False, 'contains(fashion)': False, 'contains(social)': False, 'contains(1999)': False, 'contains(news)': False, 'contains(hair)': False, 'contains(dance)': False, 'contains(innocent)': False, 'contains(camp)': False, 'contains(teacher)': False, 'contains(became)': False, 'contains(sad)': False, 'contains(wi

In [47]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [48]:
print(nltk.classify.accuracy(classifier, test_set))

0.84


In [49]:
classifier.show_most_informative_features(5)

Most Informative Features
   contains(outstanding) = True              pos : neg    =     14.0 : 1.0
   contains(wonderfully) = True              pos : neg    =      7.4 : 1.0
        contains(seagal) = True              neg : pos    =      7.4 : 1.0
         contains(mulan) = True              pos : neg    =      7.0 : 1.0
         contains(damon) = True              pos : neg    =      6.1 : 1.0


In [51]:
from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1

In [52]:
common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(100)]
print(common_suffixes)


['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his', 'th', 'll', 'le', 'ce', 'by', 'ts', 'me', 've', "'", 'se', 'ut', 'was', 'for', 'ent', 'ch', 'k', 'w', 'ld', '`', 'rs', 'ted', 'ere', 'her', 'ne', 'ns', 'ith', 'ad', 'ry', ')', '(', 'te', '--', 'ay', 'ty', 'ot', 'p', 'nce', "'s", 'ter', 'om', 'ss', ':', 'we', 'are', 'c', 'ers', 'uld', 'had', 'so', 'ey']


In [58]:
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
        features['endswith({})'.format(suffix)] = word.lower().endswith(suffix)
    return features

In [59]:
tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n,g) in tagged_words]


In [60]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.DecisionTreeClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.6270512182993535

In [62]:
classifier.classify(pos_features('cats'))

'NNS'

In [64]:
print(test_set[1])

({'endswith(e)': False, 'endswith(,)': False, 'endswith(.)': False, 'endswith(s)': False, 'endswith(d)': False, 'endswith(t)': False, 'endswith(he)': False, 'endswith(n)': True, 'endswith(a)': False, 'endswith(of)': False, 'endswith(the)': False, 'endswith(y)': False, 'endswith(r)': False, 'endswith(to)': False, 'endswith(in)': False, 'endswith(f)': False, 'endswith(o)': False, 'endswith(ed)': False, 'endswith(nd)': False, 'endswith(is)': False, 'endswith(on)': True, 'endswith(l)': False, 'endswith(g)': False, 'endswith(and)': False, 'endswith(ng)': False, 'endswith(er)': False, 'endswith(as)': False, 'endswith(ing)': False, 'endswith(h)': False, 'endswith(at)': False, 'endswith(es)': False, 'endswith(or)': False, 'endswith(re)': False, 'endswith(it)': False, 'endswith(``)': False, 'endswith(an)': False, "endswith('')": False, 'endswith(m)': False, 'endswith(;)': False, 'endswith(i)': False, 'endswith(ly)': False, 'endswith(ion)': False, 'endswith(en)': False, 'endswith(al)': False, 'e

In [67]:
def pos_features(sentence, i):
    features = {"suffix(1)": sentence[i][-1:],
                "suffix(2)": sentence[i][-2:],
                "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
    return features

In [68]:
pos_features(brown.sents()[0], 8)

{'suffix(1)': 'n', 'suffix(2)': 'on', 'suffix(3)': 'ion', 'prev-word': 'an'}

In [ ]:
tagged_sents = brown.tagged_sents(categories='news')
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
        featuresets.append( (pos_features(untagged_sent, i), tag) )

In [69]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

nltk.classify.accuracy(classifier, test_set)

0.5636001989060169

In [70]:
print(test_set[0])

({'endswith(e)': True, 'endswith(,)': False, 'endswith(.)': False, 'endswith(s)': False, 'endswith(d)': False, 'endswith(t)': False, 'endswith(he)': True, 'endswith(n)': False, 'endswith(a)': False, 'endswith(of)': False, 'endswith(the)': True, 'endswith(y)': False, 'endswith(r)': False, 'endswith(to)': False, 'endswith(in)': False, 'endswith(f)': False, 'endswith(o)': False, 'endswith(ed)': False, 'endswith(nd)': False, 'endswith(is)': False, 'endswith(on)': False, 'endswith(l)': False, 'endswith(g)': False, 'endswith(and)': False, 'endswith(ng)': False, 'endswith(er)': False, 'endswith(as)': False, 'endswith(ing)': False, 'endswith(h)': False, 'endswith(at)': False, 'endswith(es)': False, 'endswith(or)': False, 'endswith(re)': False, 'endswith(it)': False, 'endswith(``)': False, 'endswith(an)': False, "endswith('')": False, 'endswith(m)': False, 'endswith(;)': False, 'endswith(i)': False, 'endswith(ly)': False, 'endswith(ion)': False, 'endswith(en)': False, 'endswith(al)': False, 'en

In [80]:
sents = nltk.corpus.treebank_raw.sents()
tokens = []
boundaries = set()
offset = 0
for sent in sents:
    tokens.extend(sent)
    offset += len(sent)
    boundaries.add(offset-1)


In [81]:
def punct_features(tokens, i):
    return {'next-word-capitalized': tokens[i+1][0].isupper(),
    'prev-word': tokens[i-1].lower(),
    'punct': tokens[i],
    'prev-word-is-one-char': len(tokens[i-1]) == 1}

In [86]:
featuresets = [(punct_features(tokens, i), (i in boundaries)) for i in range(1, len(tokens)-1) if (tokens[i] in '.?!')]

In [87]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.936026936026936

In [88]:
print(train_set[0])

({'next-word-capitalized': True, 'prev-word': 'popular', 'punct': '.', 'prev-word-is-one-char': False}, True)


In [89]:
posts = nltk.corpus.nps_chat.xml_posts()[:10000]

In [91]:
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

In [92]:
featuresets = [(dialogue_act_features(post.text), post.get('class')) for post in posts]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.667

In [93]:
print(test_set[0])

({'contains(now)': True, 'contains(im)': True, 'contains(left)': True, 'contains(with)': True, 'contains(this)': True, 'contains(gay)': True, 'contains(name)': True}, 'Statement')
